# Access Control - Creating an \<ACP> Resource

This notebook will show how to create an \<ACP> resource, and how to assigns it to a resource.

- How does Authorization with \<ACP>'s Work?
- CREATE an \<ACP> resource
- Link the \<ACP> to the \<container> resource
- Make request to resources from another originator


## Intitialization
<div class="alert alert-block alert-info" style="border-radius: 10px;">
This notebook relies on the resources created in the exercise's previous notebooks.
<br/>

The following section imports and initializes necessary functions and configurations. 
It must be executed once for each notebook before running any of cells.</div>

In [ ]:
%run ../src/init.py

## Bob tries to retrieve Alice's \<container> resource

Let's first try Bob to access Alice's \<container> :

In [ ]:
RETRIEVE (                                                   # RETRIEVE request
    target                  = f'{cseRN}/aeAlice/container',  # Target resource is Alice's <container>
    originator              = 'CBob',                        # Request's originator is Bob (mandatory)
    requestIdentifier       = '123',                         # Request identifier (mandatory)
    releaseVersionIndicator = '3'                            # Request's release version indicator (mandatory)
)

The request **fails** because Bob has no permissions to the \<container> resource, and he is also not the original creator of that resource.

Let's try to fix this with the following requests.




## Create an \<ACP> resource

In this exercise we will create an \<ACP> resource under the \<AE> *aeAlice*.

An \<ACP> resource contains mainly a number of *access control rules* for managing access to other resources and to itself. These rules are used in a match-making procedures to control the access to the resources.

Access control rules may contain a number of parameters, like time windows, IP address ranges etc, but in our exercises we will only use the originator and the the request operation type for checking.

In [ ]:
CREATE (                                            # CREATE request
    target                  = f'{cseRN}/aeAlice',   # Create the <ACP> under aeAlice 
    originator              = 'CAlice',             # Request's originator (mandatory)
    requestIdentifier       = '123',                # Request identifier (mandatory)
    releaseVersionIndicator = '3',                  # Request's release version indicator (mandatory)
    resourceType            = '1',                  # The request creates an <ACP> resource
    
    # Request Content
    content = {
        'm2m:acp': {
            'rn': 'acp',                            # Give the resource the name 'acp'
            'pv': {                                 # Set the privileges attribute
                'acr': [                            # Add access control rules to the privileges
                    {   'acor': [ 'CAlice' ],       # Add Alice to this rule
                        'acop': 63                  # Set allowed request types (63 = all possible request types)
                    },
                    {   'acor': [ 'CBob' ],         # Add another rule for Bob
                        'acop': 2                   # Allow only RETRIEVE for Bob
                    }
                ]
            },
            'pvs': {
                'acr': [                            # Set the self-privileges attribute
                    {   'acor': [ 'CAlice' ],       # Add Alice to this rule
                        'acop': 63                  # Set allowed request types (63 = all possible request types)
                    }
                ]
            }
        }
    }
)

## Link the \<container>  to the \<ACP> resource

We now need to link Alice's \<container> to the newly created \<ACP> resource. This is done by assigning the \<ACP>'s *resourceIdentifier* to the \<container>'s *accessControlPolicyIDs* (*acp*) attribute.

<div class="alert alert-block alert-success" style="border-radius: 10px">
Note, that when setting the <i>acpi</i> attribute it must be the only attribute in an UPDATE request. Also, the originator of such an UPDATE request is authorized against the <i>self-privileges</i> and not the <i>privileges</i> attribute. This rule is for security reasons. It prevents that just any originator with write permissions to a resource is allowed to change the access control assignments.
    
Also note, that this rule does not apply when creating a resource. You can always assign an <i>acpi</i> attribute with <i>resource identifiers</i> of already existing \<ACP> resources to a new resource.
</div>

In [ ]:
UPDATE (                                                      # UPDATE request
    target                  = f'{cseRN}/aeAlice/container',   # Update the <container>
    originator              = 'CAlice',                       # Request's originator (mandatory)
    requestIdentifier       = '123',                          # Request identifier (mandatory)
    releaseVersionIndicator = '3',                            # Request's release version indicator (mandatory)
    
    # Request Content
    content = {
        'm2m:cnt': {
            'acpi' : [ f'{cseRN}/aeAlice/acp' ]               # Assign the <ACP>'s resourceID 
        }
    }
)

## Bob tries again to retrieve Alice's \<container> resource

Now let's try to retrieve Alice's container, again with Bob as the originator.

In [ ]:
RETRIEVE (                                                   # RETRIEVE request
    target                  = f'{cseRN}/aeAlice/container',  # Target resource is Alice's <container>
    originator              = 'CBob',                        # Request's originator is Bob (mandatory)
    requestIdentifier       = '123',                         # Request identifier (mandatory)
    releaseVersionIndicator = '3'                            # Request's release version indicator (mandatory)
)

This time access is granted, the request **succeeds** and returns Alice's \<container>.

## Bob tries to delete Alice's \<container> resource

Next, Bob tries to delete Alice's \<container>:

In [ ]:
DELETE (                                                     # DELETE request
    target                  = f'{cseRN}/aeAlice/container',  # Target resource is Alice's <container>
    originator              = 'CBob',                        # Request's originator is Bob (mandatory)
    requestIdentifier       = '123',                         # Request identifier (mandatory)
    releaseVersionIndicator = '3'                            # Request's release version indicator (mandatory)
)

This request **fails** because Alice gave Bob only RETRIEVE permissions for the \<container>, but not DELETE permissions.